In [1]:
import pandas as pd
import os
import shutil

#folder_path_in = 'C:\\Users\\dmandree\\OneDrive - Guess Inc\\D Project\\TL_new'
folder_path_in = 'C:\\Users\\dmandree\\Downloads\\TL_new'

#folder_path_out = 'C:\\Users\\dmandree\\OneDrive - Guess Inc\\D Project\\TL_arch'
folder_path_out = 'C:\\Users\\dmandree\\Downloads\\TL_arch'

file_list = os.listdir(folder_path_in)

df = pd.DataFrame()

for file in file_list:

    file_path = os.path.join(folder_path_in, file)

    data = pd.read_excel(file_path, sheet_name='TurnoverList')

    df = pd.concat([df, data])
    
    df["Day"] = pd.to_datetime(df["Day"]).dt.date
    
    df = df.query("Company == 'Guess Kazakhstan' or Company == 'Guess CIS'")
    
    new_path = os.path.join(folder_path_out, file)
    
    if os.path.exists(new_path):        
        #try:
            os.remove(new_path)
        #except PermissionError:
        #    print("Файл используется другим процессом и не может быть удален.")
    
    shutil.move(file_path, folder_path_out)

df.columns = df.columns.str.lower()

In [2]:
df.head()

,day,store,company,open,amount,curr.,pcs,rcp,people,hours,work,comp:,open.1,amount.1,curr..1,pcs.1,rcp.1,people.1,hours.1,work.1
0,2024-04-16,"Kazakhstan, Almaty, Dostyk Plaza - ByMarciano",Guess Kazakhstan,True,254000.0,KZT,3,3,248,12.0,0.0,1/1/0001 12:00:00 AM,False,NaN,NaN,0,0,0,0.0,0.0
1,2024-04-16,"Kazakhstan, Almaty, Dostyk Plaza - Jeans",Guess Kazakhstan,True,212000.0,KZT,4,4,231,12.0,0.0,1/1/0001 12:00:00 AM,False,NaN,NaN,0,0,0,0.0,0.0
2,2024-04-16,"Kazakhstan, Almaty, Mega - Jeans",Guess Kazakhstan,True,448000.0,KZT,9,5,212,12.0,0.0,2023-04-18 00:00:00,True,800300.0,KZT,14,12,0,12.0,0.0
3,2024-04-16,"Kazakhstan, Almaty, Mega Acc - Accessories",Guess Kazakhstan,True,244800.0,KZT,3,2,180,12.0,0.0,2023-04-18 00:00:00,True,0.0,NaN,0,0,2,12.0,0.0
4,2024-04-16,"Kazakhstan, Astana, Khan Shatyr - Jeans",Guess Kazakhstan,True,434000.0,KZT,6,5,286,13.0,0.0,2023-04-18 00:00:00,True,90000.0,KZT,3,3,104,13.0,0.0


In [3]:
# Группировка по столбцам "дата" и "магазин" и объединение значений в одну строку
unique_combinations = df['day'].unique()
# unique_combinations = df.groupby(['Day', 'Store']).apply(lambda x: f"{x.name[0]}{x.name[1]}").unique()
outer_df = pd.DataFrame(unique_combinations)
outer_df = pd.DataFrame(unique_combinations, columns=['key'])
outer_df


,key
0,2024-04-16
1,2024-04-17


In [6]:
from sqlalchemy import create_engine
from sshtunnel import SSHTunnelForwarder
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text


# SSH tunnel configuration
ssh_tunnel = SSHTunnelForwarder(
    ('79.174.86.163', 22),
    ssh_username='root',
    ssh_password='S0SJcmYwL0ZsmUId',
    remote_bind_address=('127.0.0.1', 5432),
    local_bind_address=('127.0.0.1', 8001)
)

# Start the SSH tunnel
ssh_tunnel.start()

# Database connection configuration
params = {
    'database': 'postgres',
    'user': 'postgres',
    'password': '1296',
    'host': 'localhost',
    'port': ssh_tunnel.local_bind_port
}

# Create the SQLAlchemy engine
engine_str = f"postgresql://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['database']}"
engine = create_engine(engine_str)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Execute the query and fetch the results
query = text('select DISTINCT day as key from sales')
result = session.execute(query).fetchall()

# Read the query results into a DataFrame
inner_df = pd.read_sql(query, engine)

# Объединение DataFrame по столбцу 'key'
intersection_df = pd.merge(outer_df, inner_df, on='key', how='inner')['key'].tolist()

# Удаляем пересечения в базе данных
if intersection_df:
    delete_query = text('DELETE FROM sales WHERE day = ANY(:keys)')
    session.execute(delete_query, {'keys': intersection_df})
    session.commit()
    

# Execute the query and fetch the results

df.to_sql("sales", engine, if_exists="append", index=False)
session.commit()

# Close the session and stop the SSH tunnel
session.close()
ssh_tunnel.stop()
